In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from blueshift.api import(
                            symbol,
                            order_target_percent,
                            schedule_function,
                            date_rules,
                            time_rules,
                            get_datetime
                        )

In [ ]:
def initialize(context):
    # Define symbol
    context.security = symbol('AMZN')

    context.lookback = 200
    context.split_percentage = 0.8
    context.retrain_flag = True
    context.X_train = None
    context.X_test = None
    context.y_train = None
    context.y_test = None

    context.svc_model = None

    # Schedule the retrain_model function every month
    schedule_function(
        retrain_model,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open()
    )

    # Schedule the rebalance function to run daily at market close 
    schedule_function(
        rebalance,
        date_rule=date_rules.every_day(),
        time_rule=time_rules.market_close(minutes=1)
    )


In [ ]:
def retrain_model(context, data):
    """
        A function to retrain the  model. This function is called by
        the schedule_function in the initialize function.
    """
    context.retrain_flag = True


In [ ]:

def rebalance(context, data):
    try:
        Df = data.history(
            context.security,
            ['open', 'high', 'low', 'close', 'volume'],
            context.lookback,
            '1d')
    except IndexError:
        return

    # Create predictor variables
    Df['open-close'] = Df['open'] - Df['close']
    Df['high-low'] = Df['high'] - Df['low']

    Df = Df.dropna()

    # Store all predictor variables in a variable X
    X = Df[['open-close', 'high-low']]

    if context.retrain_flag:
        context.retrain_flag = False
        # Create target variable
        y = np.where(Df['close'].shift(-1) > Df['close'], 1, 0)

        split = int(context.split_percentage*len(Df))

        context.X_train = X[:split]
        context.y_train = y[:split]
        context.X_test = X[split:]
        context.y_test = y[split:]
        
        # Support vector classifier
        context.svc_model = SVC().fit(context.X_train, context.y_train)

    accuracy_test = accuracy_score(context.y_test, context.svc_model.predict(context.X_test))

    # Predicted Signal
    predicted_signal = context.svc_model.predict(X)[-1]

    print("{} Accuracy test: {}, Prediction: {}".format(get_datetime(), accuracy_test, predicted_signal))

     # Place the orders
    if accuracy_test > 0.5:
        if predicted_signal == 1:
            order_target_percent(context.security, 1)
        else:
            order_target_percent(context.security, 0)
    else:
        order_target_percent(context.security, 0)

In [ ]:
# order_target_percent to buy somenthing from a given balance percentage